In [2]:
#yash shinde
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import random

# Simulated spray drying dataset: [InletTemp, FeedRate, AtomSpeed] -> MoistureContent
np.random.seed(42)
X = np.random.uniform(low=150, high=200, size=(100, 3))  # features
y = 0.3*X[:,0] - 0.2*X[:,1] + 0.1*X[:,2] + np.random.normal(0, 2, 100)  # target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Neural Network Model (MLP)
def create_nn(weights):
    model = MLPRegressor(hidden_layer_sizes=(5,), max_iter=1, warm_start=True)
    model.fit(X_train, y_train)
    weights = np.array(weights)  # <-- convert to NumPy array
    i = 0
    for layer_weights in model.coefs_:
        shape = layer_weights.shape
        model.coefs_[i] = weights[:np.prod(shape)].reshape(shape)
        weights = weights[np.prod(shape):]
        i += 1
    return model

# Genetic Algorithm
def fitness_function(weights):
    model = create_nn(weights)
    preds = model.predict(X_train)
    return mean_squared_error(y_train, preds)

def crossover(p1, p2):
    point = random.randint(0, len(p1)-1)
    return p1[:point] + p2[point:]

def mutate(ind, rate=0.1):
    return [w + np.random.randn()*rate if random.random() < 0.1 else w for w in ind]

# Initialize population
n_weights = (3*5) + (5*1)  # assuming 3 input, 5 hidden, 1 output
pop = [np.random.uniform(-1, 1, n_weights).tolist() for _ in range(10)]

# Run GA
for generation in range(10):
    pop = sorted(pop, key=fitness_function)
    new_pop = pop[:2]  # elitism
    while len(new_pop) < len(pop):
        p1, p2 = random.sample(pop[:5], 2)
        child = mutate(crossover(p1, p2))
        new_pop.append(child)
    pop = new_pop
    print(f"Gen {generation+1}, MSE: {fitness_function(pop[0]):.4f}")

# Final model
best_weights = pop[0]
final_model = create_nn(best_weights)
test_preds = final_model.predict(X_test)
print("Test MSE:", mean_squared_error(y_test, test_preds))


Gen 1, MSE: 837.9926
Gen 2, MSE: 53.8717
Gen 3, MSE: 23.3819
Gen 4, MSE: 21.1217
Gen 5, MSE: 20.2816
Gen 6, MSE: 20.8011
Gen 7, MSE: 19.4869
Gen 8, MSE: 21.3119
Gen 9, MSE: 19.2535
Gen 10, MSE: 18.0603
Test MSE: 16.382767021201005


C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Admin\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_pe

In [1]:
#MK optimal mse should be less than 4

import numpy as np, random
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

np.random.seed(42)
X = (np.random.uniform(150, 200, (100, 3)) - 175) / 25
y = 0.3*X[:,0] - 0.2*X[:,1] + 0.1*X[:,2] + np.random.normal(0, 2, 100)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

def create_nn(params):
    m = MLPRegressor(hidden_layer_sizes=(5,), max_iter=2, solver='lbfgs', warm_start=True)

    m.fit(X_train, y_train)
    p = np.array(params); i = 0
    for arr in m.coefs_ + m.intercepts_:
        s = np.prod(arr.shape)
        arr[...] = p[i:i+s].reshape(arr.shape)
        i += s
    return m

def fitness(p): return mean_squared_error(y_train, create_nn(p).predict(X_train))
def crossover(p1, p2): pt = random.randint(0, len(p1)-1); return p1[:pt]+p2[pt:]
def mutate(p, r=0.1): return [w+np.random.randn()*r if random.random()<0.1 else w for w in p]

tmp = MLPRegressor(hidden_layer_sizes=(5,), max_iter=1); tmp.fit(X_train, y_train)
n_params = sum(np.prod(a.shape) for a in tmp.coefs_ + tmp.intercepts_)
pop = [np.random.uniform(-1, 1, n_params).tolist() for _ in range(10)]

for g in range(10):
    pop = sorted(pop, key=fitness)
    new_pop = pop[:2]
    while len(new_pop) < len(pop):
        c = mutate(crossover(*random.sample(pop[:5], 2)))
        new_pop.append(c)
    pop = new_pop
    print(f"Gen {g+1}, MSE: {fitness(pop[0]):.4f}")

best = pop[0]
print("Test MSE:", mean_squared_error(y_test, create_nn(best).predict(X_test)))


Gen 1, MSE: 4.4953
Gen 2, MSE: 4.2561
Gen 3, MSE: 4.2561
Gen 4, MSE: 4.2503
Gen 5, MSE: 4.2237
Gen 6, MSE: 4.2231
Gen 7, MSE: 4.2108
Gen 8, MSE: 4.2108
Gen 9, MSE: 4.2108
Gen 10, MSE: 4.2078
Test MSE: 2.549139582020622


In [ ]:
'''### Explanation of the Code:

This code implements a **genetic algorithm (GA)** to optimize the Mean Squared Error (MSE) for a neural network regression model. It uses a **Multilayer Perceptron (MLP)** regressor from `sklearn` and applies genetic algorithm techniques such as **crossover** and **mutation** to evolve a population of solutions over several generations. The aim is to minimize the MSE on the training data.

Here's a breakdown of the various components of the code:

### 1. **Data Generation and Preprocessing:**

```python
np.random.seed(42)
X = (np.random.uniform(150, 200, (100, 3)) - 175) / 25
y = 0.3*X[:,0] - 0.2*X[:,1] + 0.1*X[:,2] + np.random.normal(0, 2, 100)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
```

* **X:** The feature matrix is generated with three features, each drawn from a uniform distribution between 150 and 200, then centered around 0 with a range of `-175` and normalized by dividing by 25.
* **y:** The target variable is a linear combination of the features `X` plus some Gaussian noise (`np.random.normal(0, 2, 100)`), making the regression problem noisy.
* **Data Split:** The dataset is split into training (80%) and testing (20%) sets.

### 2. **Neural Network Model Creation (with Custom Weights):**

```python
def create_nn(params):
    m = MLPRegressor(hidden_layer_sizes=(5,), max_iter=2, solver='lbfgs', warm_start=True)
    m.fit(X_train, y_train)
    p = np.array(params)
    i = 0
    for arr in m.coefs_ + m.intercepts_:
        s = np.prod(arr.shape)
        arr[...] = p[i:i+s].reshape(arr.shape)
        i += s
    return m
```

* **MLPRegressor:** A simple neural network with one hidden layer containing 5 neurons. It is configured to use the `lbfgs` solver (an optimization method) and is trained for only 2 iterations (`max_iter=2`) to avoid overfitting.
* **Setting Weights:** The neural network weights (`m.coefs_` and `m.intercepts_`) are manually modified using the given `params` array. This allows the genetic algorithm to evolve the network weights, simulating training from scratch using evolved parameters.

### 3. **Fitness Function:**

```python
def fitness(p): return mean_squared_error(y_train, create_nn(p).predict(X_train))
```

* **Fitness Function:** This function computes the **Mean Squared Error (MSE)** on the training set using the evolved neural network model (`create_nn(p)`). The goal of the genetic algorithm is to minimize this MSE value.

### 4. **Genetic Algorithm Components:**

* **Crossover:**

```python
def crossover(p1, p2): pt = random.randint(0, len(p1)-1); return p1[:pt]+p2[pt:]
```

* This function takes two parent solutions `p1` and `p2`, and combines them by swapping parts of their parameters at a random crossover point (`pt`). This mimics biological reproduction.

* **Mutation:**

```python
def mutate(p, r=0.1): return [w+np.random.randn()*r if random.random()<0.1 else w for w in p]
```

* This function introduces small random changes (mutations) to the parameters `p` by adding random noise. The mutation rate is 10% (`random.random() < 0.1`), and the size of the noise is controlled by `r=0.1`.

### 5. **Initialization of Population:**

```python
tmp = MLPRegressor(hidden_layer_sizes=(5,), max_iter=1); tmp.fit(X_train, y_train)
n_params = sum(np.prod(a.shape) for a in tmp.coefs_ + tmp.intercepts_)
pop = [np.random.uniform(-1, 1, n_params).tolist() for _ in range(10)]
```

* **Temporary Neural Network:** A simple MLP is trained for 1 iteration to determine the number of parameters (`n_params`) in the neural network (sum of the number of elements in `coefs_` and `intercepts_`).
* **Population Initialization:** A population of size 10 is created, where each individual (solution) is represented by a vector of random values between -1 and 1, corresponding to the network weights.

### 6. **Genetic Algorithm Loop (10 Generations):**

```python
for g in range(10):
    pop = sorted(pop, key=fitness)
    new_pop = pop[:2]
    while len(new_pop) < len(pop):
        c = mutate(crossover(*random.sample(pop[:5], 2)))
        new_pop.append(c)
    pop = new_pop
    print(f"Gen {g+1}, MSE: {fitness(pop[0]):.4f}")
```

* **Selection:** The population is sorted based on the fitness values (MSE), and the top 2 individuals are selected for reproduction.
* **Crossover and Mutation:** Random pairs of individuals from the top 5 are selected, crossed over, and mutated to create new individuals. These are added to the new population.
* **Replacement:** The new population replaces the old one.
* **Generation Output:** After each generation, the fitness (MSE) of the best individual is printed.

### 7. **Final Test:**

```python
best = pop[0]
print("Test MSE:", mean_squared_error(y_test, create_nn(best).predict(X_test)))
```

* **Best Individual:** After 10 generations, the best individual (the one with the lowest MSE on the training set) is selected.
* **Test Evaluation:** The MSE of the best solution is then evaluated on the test set.

### **Key Points:**

* **Optimization Goal:** The genetic algorithm is used to minimize the MSE of the neural network model on the training data, evolving over 10 generations.
* **Evolved Parameters:** The algorithm modifies the neural network weights (parameters) through a process of selection, crossover, and mutation.
* **Evaluation:** The final solution's performance is tested on the test data, and the result is printed.

### **Theory Behind Genetic Algorithm (GA) in this Context:**

* **Selection:** The process of selecting the best individuals (neural network weights) based on their performance (MSE) on the training data.
* **Crossover:** Combining the genetic material (parameters) from two individuals to create new offspring that might inherit the best traits of both parents.
* **Mutation:** Introducing small random changes to the offspring to explore new parts of the solution space, increasing diversity in the population.
* **Survival of the Fittest:** Over generations, the population evolves to find the optimal neural network weights that minimize the MSE.

This approach can be considered an alternative to gradient-based optimization methods like backpropagation, but it is computationally more expensive and may not converge as quickly.

Would you like to discuss the results of this simulation or the algorithm in more depth?
'''